[![Buy Me a cofee](https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/coffe4.gif?raw=true ) ](https://www.buymeacoffee.com/wilzamguerrero)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;32mDrive Conected ✓')

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_Version = "V1.5" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Args = "" # @param {type:"string"}
SDZ_Dreambooth = "" #@param ["", "SDZ_New_Dreambooth", "SDZ_Old_Dreambooth"]

SDZ_Big_Models = False #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}
SDZconfigV2="--config v2-inference-v.yaml"
share = "--share --api --cors-allow-origins=https://www.painthua.com --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae" + " --ckpt=" + model_init + " --ckpt-dir=" + models_dir + " " + SDZ_Args

if not SDZ_Continue: 
  with capture.capture_output() as cap:
    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    
    clear_output()
  print('[1;96mInstalling SDZ ✓')

if SDZ_Continue:
  with capture.capture_output() as cap:
    os.chdir(SDZ_webui_path)
    clear_output()
  print('[1;96mPreparing SDZ Continue ✓')

with capture.capture_output() as cap:
  os.chdir(SDZ_webui_path)
  ############# Requirements SDZ

  import os
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  
  !pip install triton==2.0.0.dev20221202.
  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
  !git clone https://github.com/bytedance/Next-ViT.git externals/Next_ViT

  %cd models/Stable-diffusion
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/open_clip'):
    !pip install xformers==0.0.16rc425
    !pip install open-clip-torch

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt
    os.chdir(SDZ_webui_path)
    
  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
    os.chdir(SDZ_webui_path)

  else:
    Download_Base_Model == ""
    os.chdir(SDZ_webui_path)

  %cd extensions
  !git clone https://github.com/Klace/stable-diffusion-webui-instruct-pix2pix.git
  !git clone https://github.com/Extraltodeus/depthmap2mask.git
  %cd ..

  if SDZ_Dreambooth == "SDZ_New_Dreambooth":
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    
  elif SDZ_Dreambooth == "SDZ_Old_Dreambooth":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !conda install -c "xformers/label/dev" xformers -y
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    
    os.chdir(SDZ_install)
    shutil.rmtree(SDZ)
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
    %cd stable-diffusion-webui
    !git checkout 7ba3923d5b494b7756d0b12f33acb3716d830b9a
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user_old.css -O user.css
    %cd extensions
    shutil.rmtree('stable-diffusion-webui-instruct-pix2pix')
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd sd_dreambooth_extension
    !git checkout 65a4bdffc41b3ab44ec8262acce21c87d9a7fccf
    %cd ..
    %cd ..

  else:
    SDZ_Dreambooth == ""
    os.chdir(SDZ_webui_path)

  if SDZ_Big_Models:
    !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
    !sed -i "s@map_location='cpu'@map_location='cuda'@" $SDZ_webui_path'/modules/extras.py'
    
    !sed -i '902s@.*@        self.logvar = self.logvar.to(self.device)@' $SDZ_webui_path'/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py'
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' $SDZ_webui_path'/webui.py'/content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $SDZ_webui_path'/modules/sd_models.py'
  else:
    !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'
    !sed -i "s@map_location='cuda'@map_location='cpu'@" $SDZ_webui_path'/modules/extras.py'

    clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cpu"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
  !python launch.py $share
elif SDZ_Version == "V2.1":
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
  !python launch.py $share $SDZconfigV2

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/save.png?raw=true' height="50" alt="netdata"/></center>
import os
from IPython.display import clear_output

save_models_dir = "/content/sample_data"#@param {type:"string"}
os.chdir(save_models_dir)
url = "https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" #@param {type:"string"}
!wget --no-check-certificate $url

clear_output()
print('[1;32mModel Save ✓')